In [11]:
import json
import pandas as pd
import numpy as np
import os
import time
import requests

In [12]:
def get_information(method, params):
    url = 'https://api.s0.t.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content['result']

In [13]:
def getSuperCommittees():
    method = "hmy_getSuperCommittees"
    params = []
    return get_information(method, params)['current']

In [16]:
def M5_test():
    flag = True
    # get the median stake and the upper and lower level 
    result = getSuperCommittees()
    median = float(result['epos-median-stake'])
    lower = float(median- 0.15*median)
    upper = float(median + 0.15*median)
    print("median stake is " + str(median))
    print("lower bond is " + str(lower))
    print("upper bond is " + str(upper))
    
    shards = result['quorum-deciders']
    count = 0
    for k, v in shards.items():
        members = v['committee-members']
        for i in members:
            if not i['is-harmony-slot']:
                count += 1
                stake = float(i['effective-stake'])
                bls_key = i['bls-public-key']
                addr = i['earning-account']
                if stake > upper or stake < lower:
                    print("validator: ", addr, " bls public key: ", bls_key) 
                    print("effective stake is out of range. The effective stake is ", stake)
                    print("")
                    flag = False
    print("total slots verified:", count)
    if flag:
        return True
    else:
        return False

In [17]:
M5_test()

median stake is 1.364243909458443e+24
lower bond is 1.1596073230396766e+24
upper bond is 1.5688804958772096e+24
total slots verified: 320


True